In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from pathlib import Path

# Rutas corregidas
at = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/data_atento")
de = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/data_desatento")

def contar(p):
    exts = ["*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG","*.mp4","*.avi","*.MOV"]
    n = 0
    for e in exts:
        n += len(list(p.glob(e)))
    return n

print("ATENTO:", contar(at))
print("DESATENTO:", contar(de))


ATENTO: 7
DESATENTO: 7


In [ ]:
import cv2, uuid
from pathlib import Path

# Entradas: TUS videos
INPUT_AT  = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/data_atento")
INPUT_DE  = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/data_desatento")

# Salidas: frames (se crean si no existen)
OUT_AT    = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/frames_atento")
OUT_DE    = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/frames_desatento")
for d in [OUT_AT, OUT_DE]: d.mkdir(parents=True, exist_ok=True)

# Detector de rostro (Haarcascade simple)
face = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def extraer(video_path: Path, out_dir: Path, every_sec: float = 0.5) -> int:
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"[X] No abre: {video_path}")
        return 0
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    step = max(1, int(round(fps * every_sec)))

    saved, i = 0, 0
    ok, frame = cap.read()
    while ok:
        if i % step == 0:
            img = frame
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
            if len(faces):
                x,y,w,h = max(faces, key=lambda b: b[2]*b[3])
                pad = int(0.15 * max(w, h))
                x0,y0 = max(0, x - pad), max(0, y - pad)
                x1,y1 = min(frame.shape[1], x + w + pad), min(frame.shape[0], y + h + pad)
                img = frame[y0:y1, x0:x1]
            img = cv2.resize(img, (224, 224))
            cv2.imwrite(str(out_dir / f"{video_path.stem}_{uuid.uuid4().hex[:8]}.jpg"), img)
            saved += 1
        i += 1
        ok, frame = cap.read()
    cap.release()
    return saved

# Procesar todos los videos (varias extensiones)
exts = ["*.mp4","*.MP4","*.mov","*.MOV","*.avi","*.AVI"]

total_at = total_de = 0
for ext in exts:
    for v in INPUT_AT.glob(ext):
        total_at += extraer(v, OUT_AT, every_sec=0.5)
    for v in INPUT_DE.glob(ext):
        total_de += extraer(v, OUT_DE, every_sec=0.5)

print(f"Frames ATENTO: {total_at} | Frames DESATENTO: {total_de}")


Frames ATENTO: 1095 | Frames DESATENTO: 940


In [ ]:
import pandas as pd, shutil
from pathlib import Path
from sklearn.model_selection import train_test_split

FR_AT = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/frames_atento")
FR_DE = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/frames_desatento")

BASE = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/data")  # dataset final
for s in ["train","val","test"]:
    for c in ["atento","desatento"]:
        (BASE/s/c).mkdir(parents=True, exist_ok=True)

def listar(root: Path, label: str):
    exts = ["*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG"]
    rows=[]
    for e in exts:
        rows += [{"path": str(p), "label": label} for p in root.glob(e)]
    return rows

df = pd.DataFrame(listar(FR_AT, "atento") + listar(FR_DE, "desatento"))
assert not df.empty, "No hay frames en frames_atento / frames_desatento."

print(df['label'].value_counts())

# 70/15/15 estratificado
train_df, temp = train_test_split(df, test_size=0.30, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp, test_size=0.50, stratify=temp["label"], random_state=42)

def mover(split_df, split_name):
    for _, r in split_df.iterrows():
        src = Path(r.path)
        dst = BASE/split_name/r.label/src.name
        if not dst.exists():
            shutil.move(str(src), str(dst))

mover(train_df, "train"); mover(val_df, "val"); mover(test_df, "test")

print("✅ Split completo")
print("Train:", sum(1 for _ in (BASE/'train').rglob('*.jpg')))
print("Val:  ", sum(1 for _ in (BASE/'val').rglob('*.jpg')))
print("Test: ", sum(1 for _ in (BASE/'test').rglob('*.jpg')))


AssertionError: No hay frames en frames_atento / frames_desatento.

In [ ]:
# ===== Celda 5 — Entrenar MobileNetV2 (baseline + fine-tuning) =====
import os, math
from pathlib import Path
import tensorflow as tf

# --- Rutas del dataset final (de la Celda 4) ---
BASE = Path("drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/data")
IMG = 224
BATCH = 32
AUTO = tf.data.AUTOTUNE

# --- Data loaders ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    BASE/"train", image_size=(IMG, IMG), batch_size=BATCH, label_mode="binary")
val_ds = tf.keras.utils.image_dataset_from_directory(
    BASE/"val", image_size=(IMG, IMG), batch_size=BATCH, label_mode="binary")
test_ds = tf.keras.utils.image_dataset_from_directory(
    BASE/"test", image_size=(IMG, IMG), batch_size=BATCH, label_mode="binary", shuffle=False)

# Opcional: inspección rápida de balance de clases
def count_by_label(ds):
    c0 = c1 = 0
    for x, y in ds.unbatch():
        if int(y.numpy()[0]) == 0: c0 += 1
        else: c1 += 1
    return c0, c1

c0_tr, c1_tr = count_by_label(train_ds)
print(f"[Train] desatento: {c0_tr} | atento: {c1_tr}")

# --- Augmentations + preprocesamiento ---
data_aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.10),
    tf.keras.layers.RandomContrast(0.10),
    tf.keras.layers.RandomBrightness(0.10),
], name="data_augmentation")

def prep(ds, train=True):
    ds = ds.shuffle(1000) if train else ds
    ds = ds.map(lambda x,y: (tf.keras.applications.mobilenet_v2.preprocess_input(x), y),
                num_parallel_calls=AUTO)
    return ds.cache().prefetch(AUTO)

# Notas:
# - data_aug se aplica dentro del modelo (para que ocurra en GPU y en train únicamente)
train_ds_prep = prep(train_ds, train=True)
val_ds_prep   = prep(val_ds,   train=False)
test_ds_prep  = prep(test_ds,  train=False)

# --- Modelo MobileNetV2 ---
base = tf.keras.applications.MobileNetV2(
    include_top=False, weights="imagenet", input_shape=(IMG, IMG, 3))
base.trainable = False  # baseline: congelada

inputs = tf.keras.Input((IMG, IMG, 3))
x = data_aug(inputs)  # augment SOLO en entrenamiento
x = base(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.25)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy", tf.keras.metrics.AUC(name="AUC")])

# --- Callbacks ---
ckpt_path = "drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/modelos/atencion_mnv2_baseline.keras"
os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_path, monitor="val_accuracy",
                                       save_best_only=True, mode="max", verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3,
                                     restore_best_weights=True, mode="max", verbose=1)
]

# --- Entrenamiento baseline ---
EPOCHS_BASE = 20
hist_base = model.fit(train_ds_prep, validation_data=val_ds_prep,
                      epochs=EPOCHS_BASE, callbacks=callbacks)

# --- Fine-tuning: descongelar últimas capas del backbone ---
base.trainable = True
# Deja congeladas las primeras capas, libera ~las últimas 40 (ajusta si quieres)
for layer in base.layers[:-40]:
    layer.trainable = False

# Re-compilar con LR más pequeño para no destruir pesos preentrenados
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy", tf.keras.metrics.AUC(name="AUC")])

ckpt_ft_path = "drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/modelos/atencion_mnv2_finetune.keras"
callbacks_ft = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_ft_path, monitor="val_accuracy",
                                       save_best_only=True, mode="max", verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3,
                                     restore_best_weights=True, mode="max", verbose=1)
]

EPOCHS_FT = 15
hist_ft = model.fit(train_ds_prep, validation_data=val_ds_prep,
                    epochs=EPOCHS_FT, callbacks=callbacks_ft)

# --- Evaluación rápida en test ---
print("\nEvaluación en TEST:")
model.evaluate(test_ds_prep, verbose=1)

# --- Guardar modelo final ---
final_path = "drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/modelos/atencion_mnv2_final.keras"
model.save(final_path)
print(f"Modelo guardado en: {final_path}")


Found 4912 files belonging to 2 classes.
Found 931 files belonging to 2 classes.
Found 931 files belonging to 2 classes.
[Train] desatento: 2553 | atento: 2359
Epoch 1/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - AUC: 0.5231 - accuracy: 0.5224 - loss: 0.7270
Epoch 1: val_accuracy improved from -inf to 0.62728, saving model to drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/modelos/atencion_mnv2_baseline.keras
154/154 ━━━━━━━━━━━━━━━━━━━━ 43s 113ms/step - AUC: 0.5233 - accuracy: 0.5225 - loss: 0.7269 - val_AUC: 0.6764 - val_accuracy: 0.6273 - val_loss: 0.6621
Epoch 2/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - AUC: 0.6045 - accuracy: 0.5721 - loss: 0.6789
Epoch 2: val_accuracy did not improve from 0.62728
154/154 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - AUC: 0.6046 - accuracy: 0.5722 - loss: 0.6788 - val_AUC: 0.7026 - val_accuracy: 0.5016 - val_loss: 0.8773
Epoch 3/20
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - AUC: 0.6379 - accuracy: 0.5959 - loss: 0.6570
Epoch 3: val_accuracy did not 

In [ ]:
UMBRAL = 0.25  # Ajusta el umbral un poco más bajo


In [ ]:
class_weights = {0:1.0, 1:1.2}  # Aumentar un poco más el peso de "desatento"
model.fit(train_ds_prep, validation_data=val_ds_prep, epochs=EPOCHS_BASE, class_weight=class_weights)


Epoch 1/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 26s 90ms/step - accuracy: 0.7219 - loss: 0.4495 - val_accuracy: 0.7991 - val_loss: 0.4179
Epoch 2/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.7508 - loss: 0.4141 - val_accuracy: 0.7938 - val_loss: 0.4215
Epoch 3/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.7364 - loss: 0.4175 - val_accuracy: 0.7927 - val_loss: 0.4348
Epoch 4/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.7348 - loss: 0.4256 - val_accuracy: 0.7830 - val_loss: 0.4408
Epoch 5/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.7303 - loss: 0.4306 - val_accuracy: 0.8464 - val_loss: 0.3849
Epoch 6/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.7606 - loss: 0.3993 - val_accuracy: 0.8292 - val_loss: 0.3835
Epoch 7/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - accuracy: 0.7413 - loss: 0.4156 - val_accuracy: 0.8088 - val_loss: 0.4069
Epoch 8/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.7517 - loss: 0.3934 - 

In [ ]:
base.trainable = True
for layer in base.layers[:-40]:  # Puedes experimentar con un número diferente de capas
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
# === Ver clases y consistencia en cada split ===
print("train class_names:", train_ds.class_names)
print("val   class_names:", val_ds.class_names)
print("test  class_names:", test_ds.class_names)
# Nota: image_dataset_from_directory asigna 0/1 en orden alfabético.
# Con carpetas ['atento','desatento'] -> 0='atento', 1='desatento'

# === Obtener probabilidades del modelo en VAL para elegir umbral óptimo ===
import numpy as np, tensorflow as tf
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report

# Convierte dataset a arrays (VAL)
y_val = []
y_val_prob = []
for xb, yb in val_ds_prep:
    y_val.append(yb.numpy().ravel())
    y_val_prob.append(model.predict(xb, verbose=0).ravel())
y_val = np.concatenate(y_val)
y_val_prob = np.concatenate(y_val_prob)

# AUC en val
val_auc = roc_auc_score(y_val, y_val_prob)
print(f"VAL AUC: {val_auc:.3f}")

# Umbral óptimo por criterio de Youden J (tpr - fpr máximo)
fpr, tpr, thr = roc_curve(y_val, y_val_prob)
youden = tpr - fpr
best_idx = np.argmax(youden)
best_thr = thr[best_idx]
print(f"Umbral óptimo (Youden): {best_thr:.3f}")

# === Evaluar en TEST con ese umbral ===
y_test = []
y_test_prob = []
for xb, yb in test_ds_prep:
    y_test.append(yb.numpy().ravel())
    y_test_prob.append(model.predict(xb, verbose=0).ravel())
y_test = np.concatenate(y_test)
y_test_prob = np.concatenate(y_test_prob)

test_auc = roc_auc_score(y_test, y_test_prob)
print(f"TEST AUC (sin invertir): {test_auc:.3f}")

# Si AUC < 0.5, probar invertir (posible inversión de mapeo)
if test_auc < 0.5:
    y_test_prob_inv = 1.0 - y_test_prob
    test_auc_inv = roc_auc_score(y_test, y_test_prob_inv)
    print(f"TEST AUC invertido: {test_auc_inv:.3f}  (probable inversión de clases)")
    # Usar el invertido si es mejor
    use_prob = y_test_prob_inv if test_auc_inv > test_auc else y_test_prob
else:
    use_prob = y_test_prob

# Predicciones binarias con el mejor umbral de VAL
y_pred = (use_prob >= best_thr).astype(int)

# Reporte
print("\n== Clasificación (TEST) ==")
print(classification_report(y_test, y_pred, target_names=test_ds.class_names))

print("Matriz de confusión [filas=verdadero, cols=pred]:")
print(confusion_matrix(y_test, y_pred))


train class_names: ['atento', 'desatento']
val   class_names: ['atento', 'desatento']
test  class_names: ['atento', 'desatento']
VAL AUC: 0.912
Umbral óptimo (Youden): 0.206
TEST AUC (sin invertir): 0.935

== Clasificación (TEST) ==
              precision    recall  f1-score   support

      atento       0.87      0.91      0.89       480
   desatento       0.90      0.85      0.87       451

    accuracy                           0.88       931
   macro avg       0.88      0.88      0.88       931
weighted avg       0.88      0.88      0.88       931

Matriz de confusión [filas=verdadero, cols=pred]:
[[437  43]
 [ 67 384]]


In [ ]:
# Define la ruta para guardar el modelo
final_path = "drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/modelos/atencion_mnv2_final_mejorado.keras"

# Guarda el modelo
model.save(final_path)

print(f"Modelo guardado en: {final_path}")


Modelo guardado en: drive/MyDrive/UPAO/VIII/Deep Learning/Proyecto/modelos/atencion_mnv2_final_mejorado.keras
